In [13]:
import oss2
import os
import celery
import shutil

In [14]:
#常量
auth = 
bucket = 
broker=
backend = 

In [15]:
#实例化celery
cel=celery.Celery(broker=broker,backend=backend)

In [16]:
def get_score(filepath,filelist): #得到图片的分数
    results = [] 
    for i in filelist:
        with open(filepath+'/'+i, "rb") as f:
            data = f.read()
        bucket.put_object(i, data)
        results.append(cel.send_task('', args=[i],queue='', routing_key=''))
    return results

In [17]:
def sort_results(results):  #得到一个case的排名
    #得到分数以及名字
    name = []
    score = []
    for i in range(len(results)):
        name.append(results[i].get()[0])
        score.append(results[i].get()[1])
    #分数的dict
    dict_score = {}
    for i in range(len(name)):
        dict_score.update({str(name[i]):str(score[i])})
    sorted_file=sorted(dict_score.items(),key=lambda item:item[1], reverse=True)
    #姓名排名
    sort_file = []
    for i in range(len(sorted_file)):
        sort_file.append(sorted_file[i][0])
    
    return sort_file

In [18]:
def group_case(filepath): #得到case数和case的二维数组
    filelist = os.listdir(filepath)
    filelist.sort()
    listIndex = []
    i = 1
    for n in range(1,len(filelist)):
        if filelist[n-1][:11] != filelist[n][:11]:
            listIndex.append(n-1)
            i=i+1
    case = i
    #listBig = np.zeros((2,300),dtype=str)
    listBig = [([0] * 0) for i in range(case)]
    #listBig = [[0]*a]*i
    listPoint = []
    q = 0
    i = 0
    for n in range(1,len(filelist)):
        if filelist[n-1][:11] == filelist[n][:11]:
            listBig[i].append(filelist[n-1])
        else:
            listBig[i].append(filelist[n-1])
            i = i+1
    listBig[i].append(filelist[n])
    return case,listBig

In [19]:
#改名函数
def rename(filelist,path,path1,path2):
    n = 0
    for i in filelist:
        if n < 9:
            oldname = path1 + os.sep + filelist[n]
            newname = path2 + os.sep + '{}{}{}{}'.format(filelist[n][:11],'.00',str(n+1),'.MMI')
            os.rename(oldname,newname)
            print(oldname,'==>',newname)
            n=n+1
        elif n<99:
            oldname = path1 + os.sep + filelist[n]
            newname = path2 + os.sep + '{}{}{}{}'.format(filelist[n][:11],'.0',str(n+1),'.MMI')
            os.rename(oldname,newname)
            print(oldname,'==>',newname)
            n=n+1
        elif n<999:
            oldname = path1 + os.sep + filelist[n]
            newname = path2 + os.sep + '{}{}{}{}'.format(filelist[n][:11],'.',str(n+1),'.MMI')
            os.rename(oldname,newname)
            print(oldname,'==>',newname)
            n=n+1
    return 

In [20]:
path = './Desktop/test'
#文件改名
path1 = './Desktop/sort'
path2 = './Desktop/sort1'
path3 = './Desktop/results'

shutil.copytree(path1,path2)
os.makedirs(path3)

In [21]:
def sort_whole_case(path,path1,path2,path3):
    casenumber,case = group_case(path)
    for i in range(casenumber):
        result = get_score(path,case[i])
        sort_file=sort_results(result)
        cut_sort_file =[]
        for i in sort_file:
            cut_sort_file.append(i[:15]+'.MMI')
        rename(cut_sort_file,path1,path2,path3)
    return

In [22]:
sort_whole_case(path,path1,path2,path3)

./Desktop/sort1/G2008051702.004.MMI ==> ./Desktop/results/G2008051702.001.MMI
./Desktop/sort1/G2008051702.005.MMI ==> ./Desktop/results/G2008051702.002.MMI
./Desktop/sort1/G2008051702.002.MMI ==> ./Desktop/results/G2008051702.003.MMI
./Desktop/sort1/G2008051702.007.MMI ==> ./Desktop/results/G2008051702.004.MMI
./Desktop/sort1/G2008051702.008.MMI ==> ./Desktop/results/G2008051702.005.MMI
./Desktop/sort1/G2008051702.006.MMI ==> ./Desktop/results/G2008051702.006.MMI
./Desktop/sort1/G2008051702.010.MMI ==> ./Desktop/results/G2008051702.007.MMI
./Desktop/sort1/G2008051702.003.MMI ==> ./Desktop/results/G2008051702.008.MMI
./Desktop/sort1/G2008051702.009.MMI ==> ./Desktop/results/G2008051702.009.MMI
./Desktop/sort1/G2008051702.001.MMI ==> ./Desktop/results/G2008051702.010.MMI
./Desktop/sort1/L2008051702.004.MMI ==> ./Desktop/results/L2008051702.001.MMI
./Desktop/sort1/L2008051702.005.MMI ==> ./Desktop/results/L2008051702.002.MMI
./Desktop/sort1/L2008051702.002.MMI ==> ./Desktop/results/L20080

In [12]:
shutil.rmtree(path2)
shutil.rmtree(path3)